In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
from collections import deque
from keras.models import load_model
import yfinance as yf
import funzioni as fx
#====================================================================================
class TradingEnvironment:
    
    def __init__(self, data, initial_balance=1000, commission=2.0):
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.reset()
        
    def reset(self):
        self.balance = self.initial_balance
        self.position = 0
        self.current_step = 0
        self.done = False
        return self._get_state()
    
    def _get_state(self):
        max_buyable = self.balance // self.data[self.current_step, 3]
        return [self.balance, self.position, max_buyable] + self.data[self.current_step].tolist()
    
    def step(self, action, num_shares):
        prev_balance = self.balance
        price = self.data[self.current_step, 3]  # Close price
        cost = num_shares * (price + self.commission)
        
        if action == 1:  # Buy
            if cost <= self.balance:
                self.balance -= cost
                self.position += num_shares
        elif action == 2:  # Sell
            if num_shares <= self.position:
                self.balance += cost
                self.position -= num_shares
                
        self.current_step += 1
        
        if self.current_step >= len(self.data):
            self.done = True
            
        reward = self.balance - prev_balance
        state = self._get_state()
        
        return state, reward, self.done, {}

#====================================================================================
class DQNAgent:
    
    def __init__(self, state_size, action_size, num_shares_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.num_shares_size = num_shares_size
        self.model = create_agent(state_size, action_size, num_shares_size)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            action = random.randrange(self.action_size)
            num_shares = random.randrange(1, self.num_shares_size)
            return action, num_shares
        act_values = self.model.predict(state)
        action = np.argmax(act_values[0][:self.action_size])
        num_shares = np.argmax(act_values[0][self.action_size:])
        return action, num_shares + 1  # +1 since the range starts from 1
     
    def remember(self, state, action, num_shares, reward, next_state, done):
        self.memory.append((state, action, num_shares, reward, next_state, done))
        
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, num_shares, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)
#====================================================================================
def create_agent(state_size, action_size, num_shares_size):
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size + num_shares_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model
#====================================================================================
# Supponiamo che i dati siano in un array numpy chiamato 'data'
data = yf.download("US02319V1035", start='2015-01-01', end='2023-07-31', progress=False).values
#data = fx.crea_indicatori(data)

bilancio = 1000

env = TradingEnvironment(data, initial_balance=bilancio)
state_size = len(env.reset())
action_size = 3  # Stay, Buy, Sell

num_shares_size = bilancio // data[-1, 3]
agent = DQNAgent(state_size, action_size, num_shares_size)
batch_size = 32

for e in range(1000):  # Numero di episodi
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    print(f"Episode: {e+1}/{1000}, Score: {env.balance}")
    
    for time in range(1000):  # max time to avoid infinite loop
        #print(f"Episode: {e+1}/1000, Time: {time}/1000, Score: {env.balance}")
        action, num_shares = agent.act(state)
        next_state, reward, done, _ = env.step(action, num_shares)
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, num_shares, reward, next_state, done)
        
        state = next_state
        
        if done:
            print(f"Episode: {e+1}/{1000}, Score: {env.balance}")
            break
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

# Save the final model
agent.save("trading_model.keras")


Episode: 1/1000, Score: 1000


C:\Users\932197\AppData\Local\Temp\ipykernel_11996\40753936.py:71: DeprecationWarning:

non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version



1/1 [==============================] - 0s 35ms/step
Episode: 2/1000, Score: 1000
1/1 [==============================] - 0s 30ms/step
Episode: 3/1000, Score: 1000
1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 40ms/step
Episode: 4/1000, Score: 1000
1/1 [==============================] - 0s 26ms/step
Episode: 5/1000, Score: 1000
1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 25ms/step
Episode: 6/1000, Score: 1000
1/1 [==============================] - 0s 50ms/step
Episode: 7/1000, Score: 1000
1/1 [==============================] - 0s 37ms/step


...

1/1 [==============================] - 0s 39ms/step
Episode: 11/1000, Score: 1000
1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 40ms/step
Episode: 12/1000, Score: 1000
1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 22ms/step
Episode: 13/1000, Score: 1000
1/1 [==============================] - 0s 36ms/step
Episode: 14/1000, Score: 1000
1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 24ms/step
Episode: 15/1000, Score: 1000
1/1 [==============================] - 0s 41ms/step



KeyboardInterrupt

